In [ ]:
import numpy as np
import pickle
import numpy as np
import glob
import os
import torch
from torch import nn
from torch.nn import functional as F
from torchvision import models
import torchvision
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import *
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parallel.data_parallel import data_parallel
train_path = os.path.join(".", 'train')
train_file_list = glob.glob(os.path.join(train_path, 'images', '*.png'))
train_file_list = [f.split('/')[-1].split('.')[0] for f in train_file_list]

In [ ]:
def do_kaggle_metric(predict,truth, threshold=0.5):

    N = len(predict)
    predict = predict.reshape(N,-1)
    truth   = truth.reshape(N,-1)

    predict = predict>threshold
    truth   = truth>0.5
    intersection = truth & predict
    union        = truth | predict
    iou = intersection.sum(1)/(union.sum(1)+EPS)

    #-------------------------------------------
    result = []
    precision = []
    is_empty_truth   = (truth.sum(1)==0)
    is_empty_predict = (predict.sum(1)==0)

    threshold = np.array([0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95])
    for t in threshold:
        p = iou>=t

        tp  = (~is_empty_truth)  & (~is_empty_predict) & (iou> t)
        fp  = (~is_empty_truth)  & (~is_empty_predict) & (iou<=t)
        fn  = (~is_empty_truth)  & ( is_empty_predict)
        fp_empty = ( is_empty_truth)  & (~is_empty_predict)
        tn_empty = ( is_empty_truth)  & ( is_empty_predict)

        p = (tp + tn_empty) / (tp + tn_empty + fp + fp_empty + fn)

        result.append( np.column_stack((tp,fp,fn,tn_empty,fp_empty)) )
        precision.append(p)

    result = np.array(result).transpose(1,2,0)
    precision = np.column_stack(precision)
    precision = precision.mean(1)

    return precision, result, threshold


In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tqdm import tqdm_notebook
truths=[np.array(load_img("./train/masks/{}.png".format(idx), grayscale=True))/255 for idx in tqdm_notebook(train_file_list)]

In [ ]:
preds=[np.array(load_img("./0859_train/{}_pred.png".format(idx), grayscale=True)) / 255 for idx in tqdm_notebook(train_file_list)]

In [ ]:
truths=np.array(truths)
preds=np.array(preds)

In [ ]:
EPS=1e-9
means=[]
stds=[]
for i in np.arange(0.42, 0.8, (0.48-0.42)/50):
    a=do_kaggle_metric(preds,truths, threshold=i)[0]
    means.append(a.mean())
    stds.append(a.std())
    print("Mean: {}, Std {}, Threshold, {}".format(round(a.mean(),5),round(a.std(),5),round(i,5)))